In [1]:
!pip install pandas 


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd

In [3]:
wine_df = pd.read_csv('winequality-red.csv', sep=";")
print(f"wine_df.shape: {wine_df.shape}")

wine_df.head()

wine_df.shape: (1599, 12)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
wine_df.quality.unique()

array([5, 6, 7, 4, 8, 3])

In [5]:
# PERFORM A PREPROCESSING OF THE DATASET

In [5]:
y = wine_df.quality
X = wine_df.drop(columns = ['quality'])

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [24]:
X_train.shape, X_test.shape

((1071, 11), (528, 11))

In [6]:

X.shape, y.shape

((1599, 11), (1599,))

In [ ]:
# y in [0, 1, .., 10]

# yk = 8
# yyk = [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
#       [0.1, 0.3, 0.5, 0, 0, 0, 0, 0, 0, 0]

In [8]:
#from tf.keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import to_categorical

WINE_QUALITIES = 11

yy = to_categorical(y, num_classes=WINE_QUALITIES)

print("yy.shape", yy.shape)
yy[0]

yy.shape (1599, 11)


array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32)

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def make_model(input_shape, num_classes=WINE_QUALITIES):
    model = keras.Sequential([
        keras.Input(shape=input_shape),
        layers.Dense(48, activation="relu"),
        layers.Dense(24, activation="relu"),
        layers.Dense(WINE_QUALITIES, activation="softmax"),
    ])

    return model

In [10]:
features = X.shape[1:]
model = make_model(input_shape=features)
#keras.utils.plot_model(model, show_shapes=True)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 48)                576       
                                                                 
 dense_1 (Dense)             (None, 24)                1176      
                                                                 
 dense_2 (Dense)             (None, 11)                275       
                                                                 
Total params: 2027 (7.92 KB)
Trainable params: 2027 (7.92 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2023-11-27 22:45:41.881449: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-27 22:45:41.981119: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [11]:
epochs = 512

checkpoint_filepath = './checkpoints/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_categorical_crossentropy',
    mode='max',
    save_best_only=True)

callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    model_checkpoint_callback,
    tf.keras.callbacks.ProgbarLogger(count_mode="steps"),
    #tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, min_delta=10**-3)
]

metrics = [
    #tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.CategoricalCrossentropy(),
    tf.keras.metrics.Precision(),
    tf.keras.metrics.AUC(multi_label=True),
    #tf.keras.metrics.Recall()
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=metrics
)

In [12]:
model.fit(
    X,
    yy,
    epochs=epochs,
    callbacks=callbacks,
    validation_split = 0.2,
)

Epoch 1/512
40/40 [==============================] - 3s 24ms/step - loss: 2.9334 - categorical_crossentropy: 2.9334 - precision: 0.4288 - auc: 0.3130 - val_loss: 1.2411 - val_categorical_crossentropy: 1.2411 - val_precision: 0.5752 - val_auc: 0.3442
Epoch 2/512
40/40 [==============================] - 0s 3ms/step - loss: 1.2391 - categorical_crossentropy: 1.2391 - precision: 0.5888 - auc: 0.3353 - val_loss: 1.1906 - val_categorical_crossentropy: 1.1906 - val_precision: 0.6069 - val_auc: 0.3534
Epoch 3/512
40/40 [==============================] - 0s 3ms/step - loss: 1.1718 - categorical_crossentropy: 1.1718 - precision: 0.5901 - auc: 0.3287 - val_loss: 1.1951 - val_categorical_crossentropy: 1.1951 - val_precision: 0.5097 - val_auc: 0.3565
Epoch 4/512
40/40 [==============================] - 0s 3ms/step - loss: 1.1688 - categorical_crossentropy: 1.1688 - precision: 0.5526 - auc: 0.3259 - val_loss: 1.1865 - val_categorical_crossentropy: 1.1865 - val_precision: 0.5249 - val_auc: 0.3438
Epo

In [13]:
sample = wine_df.sample(n=5)

sample

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
122,7.3,0.695,0.00,2.5,0.075,3.0,13.0,0.99800,3.49,0.52,9.2,5
401,7.7,0.260,0.30,1.7,0.059,20.0,38.0,0.99490,3.29,0.47,10.8,6
281,7.7,0.270,0.68,3.5,0.358,5.0,10.0,0.99720,3.25,1.08,9.9,7
1042,8.9,0.500,0.21,2.2,0.088,21.0,39.0,0.99692,3.33,0.83,11.1,6
508,10.0,0.590,0.31,2.2,0.090,26.0,62.0,0.99940,3.18,0.63,10.2,6


In [14]:
x_sample, y_sample = sample.drop(columns=['quality']), sample.quality

In [15]:
y_pred = model.predict(x_sample)

y_pred

1/1 [==============================] - 0s 70ms/step


array([[6.67727429e-10, 1.03000483e-10, 7.80923104e-11, 1.12650776e-02,
        1.25128804e-02, 9.27373827e-01, 4.87271324e-02, 1.20322904e-04,
        7.50407992e-07, 1.99072807e-11, 1.44064233e-10],
       [8.14634249e-10, 2.50833965e-10, 1.65636338e-09, 4.21598187e-08,
        6.64038304e-03, 1.56934485e-01, 6.04086459e-01, 2.28017002e-01,
        4.32161056e-03, 6.37558140e-09, 5.38362029e-11],
       [2.24672672e-08, 9.26198584e-09, 1.94497596e-09, 1.55766372e-06,
        2.57481318e-02, 2.27047428e-02, 1.97119683e-01, 7.49701142e-01,
        4.72469069e-03, 4.33237846e-09, 8.50952420e-10],
       [1.40074108e-09, 4.59079275e-10, 2.10142215e-09, 5.53552049e-08,
        3.67854140e-03, 6.00024685e-02, 4.93906230e-01, 4.19808865e-01,
        2.26038508e-02, 1.48333505e-08, 1.79429291e-10],
       [2.09830886e-09, 7.11775916e-11, 5.35510025e-10, 3.00432043e-06,
        9.46861878e-03, 3.26874614e-01, 6.30013824e-01, 3.31623219e-02,
        4.77684080e-04, 4.66773153e-09, 6.21343185e-

In [20]:
y_pred[0]

array([6.67727429e-10, 1.03000483e-10, 7.80923104e-11, 1.12650776e-02,
       1.25128804e-02, 9.27373827e-01, 4.87271324e-02, 1.20322904e-04,
       7.50407992e-07, 1.99072807e-11, 1.44064233e-10], dtype=float32)

In [19]:
import numpy as np

np.argmax(y_pred, axis=1)

array([5, 6, 7, 6, 6])